<a href="https://colab.research.google.com/github/zhangxs131/Transformers_example/blob/main/%E4%B8%AD%E6%96%87%E6%83%85%E6%84%9F%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#中文情感分析（文本分类）
本文主要是使用huggingface的实战应用

*    model_checkpoint为 bert-base-chinese
*    dataset    为 datasets中的seamew/ChnSentiCorp 主要为酒店，产品评价


导入transfromers 和 datasets包

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 30.3 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 46.5 MB/s 
     |████████████████████████████████| 596 kB 48.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 44.5 MB/s 
     |████████████████████████████████| 243 kB 53.5 MB/s 
     |████████████████████████████████| 133 kB 53.3 MB/s 
     |████████████████████████████████| 144 kB 50.2 MB/s 
     |████████████████████████████████| 94 kB 491 kB/s 
     |████████████████████████████████| 271 kB 51.4 MB/s 


#构建数据集

这里实现Dataset类，继承torch.utils.data.DataSet
实现，init,__len__,__getitem__函数

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import Dataset

class SentiDataset(Dataset):
  def __init__(self,split):
    self.dataset=load_dataset(path='seamew/ChnSentiCorp',split=split)

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self,i):
    text=self.dataset[i]['text']
    label=self.dataset[i]['label']
    return text, label

dataset=SentiDataset('train')
print(len(dataset))
dataset[0]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset chn_senti_corp downloaded and prepared to /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85. Subsequent calls will reuse this data.
9600


('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
 1)

#加载tokenizer

这里bertTokenizer，预训练模型bert-base-chinese

In [ ]:
from transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained('bert-base-chinese')


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

#定义批处理函数collate_fn

首先得到text，label的list
然后使用tokenizer，进行分词，truncation截断，padding，返回tensor，设置max_length
得到nput_ids(编码后数字），attention_mask（padding部分为0，其他为1）

In [ ]:
def collate_fn(data):
  sents=[i[0] for i in data]
  labels=[i[1] for i in data]

  data=tokenizer.batch_encode_plus(batch_text_or_text_pairs=sents,
                   truncation=True,
                   padding='max_length',
                   max_length=500,
                   return_tensors='pt',
                   return_length=True)
  labels=torch.LongTensor(labels)

  return data['input_ids'],data['attention_mask'],data['token_type_ids'],labels

#实现dataloader和模型

模型这里使用BertModel，然后后面自己接一个全连接层，将[CLS]向量拿出来进行分类

In [ ]:
from torch.utils.data import DataLoader
from transformers import BertModel

dataloader=DataLoader(
    dataset=dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True
)

for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(dataloader):
  break

print(len(dataloader))
print(input_ids.shape,attention_mask.shape,token_type_ids.shape,labels)

600
torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0])


In [ ]:
#Model
pre=BertModel.from_pretrained('bert-base-chinese')

#不需要训练，不计算梯度
for param in pre.parameters():
  param.requires_grad_(False)

out=pre(input_ids,attention_mask,token_type_ids)
print(out.last_hidden_state.shape)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 500, 768])


In [ ]:
#定义下游神经网络

class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc=torch.nn.Linear(768,2)

  def forward(self,input_ids,attention_mask,token_type_ids):
    with torch.no_grad():
      out=pre(input_ids,attention_mask,token_type_ids)

    out=self.fc(out.last_hidden_state[:,0])
    out=out.softmax(dim=1)

    return out

model=Model()


#训练模型


In [ ]:
#训练 这里只训练300个batch
from transformers import AdamW

optimizer=AdamW(model.parameters(),lr=5e-4)
criterion=torch.nn.CrossEntropyLoss()

device = 'cuda' if torch.cuda.is_available() else 'cpu'


model.train().to(device)
for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(dataloader):

  input_ids=input_ids.to(device)
  attention_mask=attention_mask.to(device)
  token_type_ids=token_type_ids.to(device)
  labels=labels.to(device)

  out=model(input_ids,attention_mask,token_type_ids)

  loss=criterion(out,labels)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  if i %5==0 :
    out=out.argmax(dim=1)
    accuarcay=(out==labels).sum().item()/len(labels)

    print(i,loss.item(),accuarcay)

  if i ==300:
    break

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


0 0.7761825919151306 0.4375
5 0.6802919507026672 0.4375
10 0.6076210141181946 0.6875
15 0.6590350270271301 0.75
20 0.6124139428138733 0.625
25 0.6314241886138916 0.75
30 0.5651098489761353 0.8125
35 0.5586031079292297 0.75
40 0.5586943030357361 0.8125
45 0.47992393374443054 0.9375
50 0.4985983073711395 0.875
55 0.5494759678840637 0.875
60 0.5883544087409973 0.75
65 0.4599294662475586 1.0
70 0.4334254562854767 0.9375
75 0.4909467399120331 0.875
80 0.5116227269172668 0.875
85 0.46637165546417236 0.9375
90 0.4713159501552582 0.875
95 0.44724246859550476 0.9375
100 0.5336244702339172 0.8125
105 0.4327835440635681 1.0
110 0.5358858108520508 0.8125
115 0.599048376083374 0.75
120 0.45633283257484436 0.9375
125 0.5189595818519592 0.875
130 0.5162624716758728 0.8125
135 0.44995495676994324 0.875
140 0.5727562308311462 0.6875
145 0.5259324312210083 0.875
150 0.4054056704044342 1.0
155 0.4792492091655731 0.8125
160 0.4741590917110443 0.875
165 0.44819962978363037 0.9375
170 0.4627234935760498 0.8

#Test

在测试集进行测试

In [ ]:
def test():
  model.eval().to(device)
  correct=0
  total=0

  loader_test=DataLoader(dataset=SentiDataset('validation'),
              batch_size=32,
              collate_fn=collate_fn,
              drop_last=True)
  
  with torch.no_grad():
    for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(loader_test):

      if i ==10:
        break

      input_ids=input_ids.to(device)
      attention_mask=attention_mask.to(device)
      token_type_ids=token_type_ids.to(device)
      labels=labels.to(device)

      out=model(input_ids,attention_mask,token_type_ids)
      out=out.argmax(dim=1)
      correct+=(out==labels).sum().item()
      total+=len(labels)



  print('test accuarcy:',correct/total)

test()

Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


0.85625
